# <font color='orange'> Numerical Exercises 7 </font>

In this exercise we want to study a statistical system of $ N_{part} = 108 $ Argon particles in the **Canonical Ensamble (NVT)** subjected to a Lennard-Jones potential

$$ 
  V_{LJ}(r) = 4 \epsilon \ \bigg[ \bigg( \frac{\sigma}{r} \bigg)^{12} - 
  \ \ \bigg( \frac{\sigma}{r} \bigg)^6 \bigg]
$$

where $ r $ is the distance between two interacting particles, $ \epsilon $ is the depth of the potential well (the "dispersion energy"), and $ \sigma $ is the distance at which the particle-particle potential energy $ V_{LJ} $ is zero (the "size of the particle").<br>
To reach this goal we follow a Statistical Mechanics approach by exploiting the Metropolis algorithm to sample the Boltzmann weight (as in <font color='orange'> Numerical Exercise 6 </font>)

$$ \frac{1}{Z} e^{- \beta H} $$

where $ Z $ is the partition function of the system (remember that this normalization goes away in the acceptance step), $ \beta = \dfrac{1}{k_B T} $ and $ H $ the (Classical) Hamiltonian, and thus calculating the thermodynamic observables of our interest as 

\begin{equation}
  O \equiv {\big \langle O \big \rangle}_{NVT} = 
  \int_V \dfrac{d_{N_{part}}\vec{r} \ d_{N_{part}}\vec{p}}{h^{3N_{part}} N_{part}!} \ O(\vec{r}, \vec{p}) \ 
  \dfrac{e^{- \beta H(\vec{r}, \vec{p})}}{Z}  \tag{$\star$}
  \label{eq:canonical_average}
\end{equation}

In particular we are interested in the potential energy per particle $ \dfrac{U}{N} = \dfrac{1}{N} {\big \langle \sum_{i < j} V_{LJ}(r_{ij}) \big \rangle}_{NVT} $ and the pressure $ P = \rho k_BT + \dfrac{1}{V} {\big \langle W \big \rangle}_{NVT} $, where $ \rho $ is the particle density of the Argon system and $ W $ is the virial, i.e.

$$ W = \frac{1}{3} \sum_{i=1}^{N_{part}} \vec{r}_i \cdot \vec{f}_i $$

with $ \vec{f}_i $ being the force acting on the particle $i$ due to the Lennard-Jones interaction. The probability distribution that we have to sample is actually simpler, i.e.

$$ \frac{1}{Z} e^{- \beta \sum_{i<j} V_{LJ}(r_{ij})} $$

as these observables are depending only on the positions of the particles, and the non configurational part of the Boltzmann weight is exactly integrated and simplifies the measure normalization in \eqref{eq:canonical_average}.

## <font color='blue'>Exercise 07.1</font>   

Before starting with the actual simulations of the solid, liquid and gas phases of Argon it is required to equilibrate the system and determine (this time in a precise manner) the length of the blocks to be used for the estimation of the observable averages and uncertainties so as to be sure to measure uncorrelated quantities in each of these blocks.<br>
First of all I try to understand after how many simulations the system has reached the thermodynamic equilibrium at the target temperature of each phase, implementing in the provided code the restart from an initial (not random) configuration generated at the end of a previous simulation. To do this I sample $ M=5 \times 10^5 $ system spatial configurations through the Metropolis algorithm and measure the instantaneous values of potential energy per particle and pressure in each of these configurations; I repeat this procedure five times starting each simulation from the final configuration generated in the previous one, apart from the very first one in which I start from a configuration as reasonable as possible for the characteristics of the phase we want to study (file <font face="Courier">config.0</font>), and graph below the curves obtained in each of them. During this equilibration phase I also fix the Metropolis acceptance ratio approximately to $50\%$ by adjusting the value of the $ \delta $ parameter.<br>
I recall the values of the parameters (superscript $ ^{\star} $ means Lennard-Jones units and $ k_B = 1 $) that characterize the three phases of our interest in the table below.

| Parameter                               | Symbol                       | Value           |
| -------- -------- -------- --------     | ----- ----- ------ ------    | --- --- --- --- |
| LJ&nbsp;dispersion&nbsp;energy&#9;      | $ \epsilon $ / $ k_B \ (K) $ | $ 120 $         |
| LJ&nbsp;particle&nbsp;size&#9;          | $ \sigma \ (nm) $            | $ 0.34 $        |
| Argon&nbsp;mass&#9;                     | $ m $ $ (amu) $              | $39$.$948$      |
| Solid&nbsp;particle&nbsp;density&#9;    | $ \rho^{\star}_{solid} $     | $ 1.1 $         |
| Liquid&nbsp;particle&nbsp;density&#9;   | $ \rho^{\star}_{liquid} $    | $ 0.8 $         |
| Gas&nbsp;particle&nbsp;density&#9;      | $ \rho^{\star}_{gas} $       | $ 0.05 $        |
| Solid&nbsp;target&nbsp;temperature&#9;  | $ T^{\star}_{solid} $        | $ 0.8 $         | 
| Liquid&nbsp;target&nbsp;temperature&#9; | $ T^{\star}_{liquid} $       | $ 1.1 $         |
| Gas&nbsp;target&nbsp;temperature&#9;    | $ T^{\star}_{gas} $          | $ 1.2 $         |
| Solid&nbsp;cut-off&nbsp;radius&#9;      | $ r_{c, solid}^{\star} $     | $ 2.2 $         |
| Liquid&nbsp;cut-off&nbsp;radius&#9;     | $ r_{c, liquid}^{\star} $    | $ 2.5 $         |
| Gas&nbsp;cut-off&nbsp;radius&#9;        | $ r_{c, gas}^{\star} $       | $ 5.0 $         |


As it can be easily seen from the graphs below it is sufficient to perform a single balancing simulation to reach the thermodynamic equilibrium of the solid and liquid phase, i.e. to eliminate the initial "stair" that appears in the graphs of $ \dfrac{U}{N} $ and $ P $ instantaneous values (I just show the first hundreds of MC steps); the system in the gas phase already seems to start from an equilibrium phase (in this case the starting configuration, the <font face="Courier">config.0</font> file, is typical of a liquid phase).
In addition, unlike the case of **Molecular Dynamics** (<font color='orange'>Numerical Exercise 4</font>), where the curves of the instantaneous values of the observables are gradually approaching the equilibrium restart after restart, in this case the curves show that the equilibrium is reached just after a few hundred MC-steps, without the need to resort to a possible restart. In my opinion this different behavior is to be attributed to the different role that the temperature plays in the two different algorithms, as discussed at the end of the notebook. <br>
Based on everything that has been outlined so far, I will always perform $ N_{eq} = 200 $ MC equilibration steps in which I move the system without making measurements in order to eliminate the initial "stair" before starting with the actual simulation, unless  otherwise indicated.
This should be sufficient to equilibrate all three phases.

In [1]:
#Modules
from math import sqrt, exp
import numpy as np
from scipy.optimize import curve_fit
from scipy.special import expit
import matplotlib
import matplotlib.pyplot as plt
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<a href="javascript:code_toggle()"> Show/Hide code cells </a>.''')

In [2]:
#In the case of single graphs in the figure
#I set the central alignment
HTML("""
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>
""")

In [ ]:
###########################################
#choice of how many simulations are
#needed to equilibrate the system
###########################################
#load the data
M=5*10**5  #Monte Carlo steps
epot_solid_0=np.loadtxt("07.1/solid/epotential_equilibration.dat", skiprows=1, max_rows=M)
epot_solid_1=np.loadtxt("07.1/solid/epotential_equilibration.dat", skiprows=M+1, max_rows=M)
epot_solid_2=np.loadtxt("07.1/solid/epotential_equilibration.dat", skiprows=2*(M+1), max_rows=M)
epot_solid_3=np.loadtxt("07.1/solid/epotential_equilibration.dat", skiprows=3*(M+1), max_rows=M)
epot_solid_4=np.loadtxt("07.1/solid/epotential_equilibration.dat", skiprows=4*(M+1), max_rows=M)
press_solid_0=np.loadtxt("07.1/solid/pressure_equilibration.dat", skiprows=1, max_rows=M)
press_solid_1=np.loadtxt("07.1/solid/pressure_equilibration.dat", skiprows=M+1, max_rows=M)
press_solid_2=np.loadtxt("07.1/solid/pressure_equilibration.dat", skiprows=2*(M+1), max_rows=M)
press_solid_3=np.loadtxt("07.1/solid/pressure_equilibration.dat", skiprows=3*(M+1), max_rows=M)
press_solid_4=np.loadtxt("07.1/solid/pressure_equilibration.dat", skiprows=4*(M+1), max_rows=M)

epot_liquid_0=np.loadtxt("07.1/liquid/epotential_equilibration.dat", skiprows=1, max_rows=M)
epot_liquid_1=np.loadtxt("07.1/liquid/epotential_equilibration.dat", skiprows=M+1, max_rows=M)
epot_liquid_2=np.loadtxt("07.1/liquid/epotential_equilibration.dat", skiprows=2*(M+1), max_rows=M)
epot_liquid_3=np.loadtxt("07.1/liquid/epotential_equilibration.dat", skiprows=3*(M+1), max_rows=M)
epot_liquid_4=np.loadtxt("07.1/liquid/epotential_equilibration.dat", skiprows=4*(M+1), max_rows=M)
press_liquid_0=np.loadtxt("07.1/liquid/pressure_equilibration.dat", skiprows=1, max_rows=M)
press_liquid_1=np.loadtxt("07.1/liquid/pressure_equilibration.dat", skiprows=M+1, max_rows=M)
press_liquid_2=np.loadtxt("07.1/liquid/pressure_equilibration.dat", skiprows=2*(M+1), max_rows=M)
press_liquid_3=np.loadtxt("07.1/liquid/pressure_equilibration.dat", skiprows=3*(M+1), max_rows=M)
press_liquid_4=np.loadtxt("07.1/liquid/pressure_equilibration.dat", skiprows=4*(M+1), max_rows=M)

epot_gas_0=np.loadtxt("07.1/gas/epotential_equilibration.dat", skiprows=1, max_rows=M)
epot_gas_1=np.loadtxt("07.1/gas/epotential_equilibration.dat", skiprows=M+1, max_rows=M)
epot_gas_2=np.loadtxt("07.1/gas/epotential_equilibration.dat", skiprows=2*(M+1), max_rows=M)
epot_gas_3=np.loadtxt("07.1/gas/epotential_equilibration.dat", skiprows=3*(M+1), max_rows=M)
epot_gas_4=np.loadtxt("07.1/gas/epotential_equilibration.dat", skiprows=4*(M+1), max_rows=M)
press_gas_0=np.loadtxt("07.1/gas/pressure_equilibration.dat", skiprows=1, max_rows=M)
press_gas_1=np.loadtxt("07.1/gas/pressure_equilibration.dat", skiprows=M+1, max_rows=M)
press_gas_2=np.loadtxt("07.1/gas/pressure_equilibration.dat", skiprows=2*(M+1), max_rows=M)
press_gas_3=np.loadtxt("07.1/gas/pressure_equilibration.dat", skiprows=3*(M+1), max_rows=M)
press_gas_4=np.loadtxt("07.1/gas/pressure_equilibration.dat", skiprows=4*(M+1), max_rows=M)

In [ ]:
#plots
#Solid Phase
plt.figure(figsize=[70, 25])  #deafult (6.4,4.8)
plt.suptitle('Equilibration of the Solid Phase', y=1.05, fontname = 'Bradley Hand', fontsize=110)

plt.subplot(1, 2, 1)
plt.title('instantaneous Potential Energy', fontname = 'Bradley Hand', fontsize=90)
plt.xlabel('$step$', fontsize=80, labelpad=50)
plt.ylabel('$\dfrac{U}{N}$', fontsize=90, labelpad=50)
plt.xticks(fontsize=60)
plt.yticks(fontsize=60)
plt.plot(epot_solid_0[:500], label='Start Simulation')
plt.plot(epot_solid_1[:500], label='1st Restart')
plt.plot(epot_solid_2[:500], label='2nd Restart')
plt.plot(epot_solid_3[:500], label='3rd Restart')
plt.plot(epot_solid_4[:500], label='4th Restart')
plt.legend(fontsize=60, loc='best')

plt.subplot(1, 2, 2)
plt.title('Instantaneous Pressure', fontname = 'Bradley Hand', fontsize=90)
plt.xlabel('$step$', fontsize=80, labelpad=50)
plt.ylabel('$P$', fontsize=90, labelpad=50)
plt.xticks(fontsize=60)
plt.yticks(fontsize=60)
plt.plot(press_solid_0[:500], label='Start Simulation')
plt.plot(press_solid_1[:500], label='1st Restart')
plt.plot(press_solid_2[:500], label='2nd Restart')
plt.plot(press_solid_3[:500], label='3rd Restart')
plt.plot(press_solid_4[:500], label='4th Restart')
plt.legend(fontsize=60, loc='best')

plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.3, hspace=0.3)
plt.show()
print('\n\n')

#Liquid Phase
plt.figure(figsize=[70, 25])  #deafult (6.4,4.8)
plt.suptitle('Equilibration of the Liquid Phase', y=1.05, fontname = 'Bradley Hand', fontsize=110)

plt.subplot(1, 2, 1)
plt.title('Instantaneous Potential Energy', fontname = 'Bradley Hand', fontsize=90)
plt.xlabel('$step$', fontsize=80, labelpad=50)
plt.ylabel('$\dfrac{U}{N}$', fontsize=90, labelpad=50)
plt.xticks(fontsize=60)
plt.yticks(fontsize=60)
plt.plot(epot_liquid_0[:500], label='Start Simulation')
plt.plot(epot_liquid_1[:500], label='1st Restart')
plt.plot(epot_liquid_2[:500], label='2nd Restart')
plt.plot(epot_liquid_3[:500], label='3rd Restart')
plt.plot(epot_liquid_4[:500], label='4th Restart')
plt.legend(fontsize=60, loc='best')

plt.subplot(1, 2, 2)
plt.title('instantaneous Pressure', fontname = 'Bradley Hand', fontsize=90)
plt.xlabel('$step$', fontsize=80, labelpad=50)
plt.ylabel('$P$', fontsize=90, labelpad=50)
plt.xticks(fontsize=60)
plt.yticks(fontsize=60)
plt.plot(press_liquid_0[:500], label='Start Simulation')
plt.plot(press_liquid_1[:500], label='1st Restart')
plt.plot(press_liquid_2[:500], label='2nd Restart')
plt.plot(press_liquid_3[:500], label='3rd Restart')
plt.plot(press_liquid_4[:500], label='4th Restart')
plt.legend(fontsize=60, loc='best')

plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.3, hspace=0.3)
plt.show()
print('\n\n')


#Gas Phase
plt.figure(figsize=[70, 25])  #deafult (6.4,4.8)
plt.suptitle('Equilibration of the Gas Phase', y=1.05, fontname = 'Bradley Hand', fontsize=110)

plt.subplot(1, 2, 1)
plt.title('Instantaneous Potential Energy', fontname = 'Bradley Hand', fontsize=90)
plt.xlabel('$step$', fontsize=80, labelpad=50)
plt.ylabel('$\dfrac{U}{N}$', fontsize=90, labelpad=50)
plt.xticks(fontsize=60)
plt.yticks(fontsize=60)
plt.plot(epot_gas_0[:300], label='Start Simulation')
plt.plot(epot_gas_1[:300], label='1st Restart')
plt.plot(epot_gas_2[:300], label='2nd Restart')
plt.plot(epot_gas_3[:300], label='3rd Restart')
plt.plot(epot_gas_4[:300], label='4th Restart')
plt.legend(fontsize=60, loc='best')

plt.subplot(1, 2, 2)
plt.title('instantaneous Pressure', fontname = 'Bradley Hand', fontsize=90)
plt.xlabel('$step$', fontsize=80, labelpad=50)
plt.ylabel('$P$', fontsize=90, labelpad=50)
plt.xticks(fontsize=60)
plt.yticks(fontsize=60)
plt.plot(press_gas_0[:300], label='Start Simulation')
plt.plot(press_gas_1[:300], label='1st Restart')
plt.plot(press_gas_2[:300], label='2nd Restart')
plt.plot(press_gas_3[:300], label='3rd Restart')
plt.plot(press_gas_4[:300], label='4th Restart')
plt.legend(fontsize=60, loc='best')

plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.3, hspace=0.3)
plt.show()

Equilibrated the three phases, I use the $ M $ instantaneous values of $ \dfrac{U}{N} $ and $ P $ to calculate the respective *time-displaced autocorrelation functions* and estimate the **correlation MC time** $ \tau_c $. In a Monte Carlo simulation the time, which is discrete, is measured in terms of MC steps; made this specification, the autocorrelation function that I want to compute is:

$$
  A_{C_{[O]}}(t) = \dfrac{\bigg(\dfrac{1}{t_{max}-t} \sum_{t' = 0}^{ t_{max} - t } O(t') O(t' + t)\bigg)
                 - \bigg(\dfrac{1}{t_{max}-t} \sum_{t' = 0}^{t_{max}-t} O(t') \bigg) 
                 \bigg( \dfrac{1}{t_{max}-t} \sum_{t' = 0}^{t_{max}-t} O(t' + t) \bigg)}
                 {\bigg(\dfrac{1}{t_{max}} \sum_{t' = 0}^{t_{max}} O^{2}(t') \bigg) - 
                 \bigg(\dfrac{1}{t_{max}} \sum_{t' = 0}^{t_{max}} O(t') \bigg)^2}
$$

with $ t_{max} = M $ and $ O = \bigg\{ \dfrac{U}{N}; P \bigg\} $. The correlation time $ \tau_c $ is the typical time-scale on which the autocorrelation drops off; in fact the autocorrelation is expected to fall off exponentially at long times as 

$$ A_{C_{[O]}}(t) \sim e^{-t / \tau_c} $$

I show below my results with respect to this analysis of the correlation time during the Metropolis sampling.
Specifically, I compute the autocorrelation functions up to time $ t = 1000 $ and I try to extrapolate a correlation time via an exponential fit $ f(t) = ae^{-bt} + c $, i.e. $ ( \tau_c )_{fit} \sim \frac{1}{b} $.

In [ ]:
###########################################
#Time-displaced Autocorrelation Function
###########################################
#Functions
#Statistical covariance of the RV
#with a time-delayed copy of itself
def Cov_t(x, t_max, t, case):  
    accumulate = 0
    
    if case == 0:
        for t_prime in range(0, t_max):
            accumulate += x[t_prime]*x[t_prime]
    else:  #for example case = 1
        for t_prime in range(0, (t_max-t)):
            accumulate += x[t_prime]*x[t_prime+t]
    return accumulate

def mean_t(x, t_max, t, case):
    accumulate = 0
    
    if case == 0:
        for t_prime in range(0, (t_max-t)):
            accumulate += x[t_prime]
    elif case == 1:
        for t_prime in range(0, t_max):
            accumulate += x[t_prime]
    else:  #for example case = 2
        for t_prime in range(0, (t_max-t)):
            accumulate += x[t_prime+t]
    
    return accumulate

#Function for statistical uncertainty estimation
def error(ave, ave2, n):  
    if n==0:
        return 0
    else:
        return sqrt((ave2[n] - ave[n]**2)/n)
    
#Definition of the Fitting Function
def exp_fit(t, a, b, c):
    y = a*expit(-b*t) + c
    return y
    

#Constants 
M = 5*10**5  #number of Monte Carlo steps
             #this corresponds to the t_max (discrete)
times = np.arange(0, 1000)  #time points for the calculation
                            #of the autocorrelation function

In [ ]:
#load the data
e_solid = np.loadtxt("07.1/solid/epot.dat", skiprows=1)
p_solid = np.loadtxt("07.1/solid/pres.dat", skiprows=1)
e_liquid = np.loadtxt("07.1/liquid/epot.dat", skiprows=1)
p_liquid = np.loadtxt("07.1/liquid/pres.dat", skiprows=1)
e_gas = np.loadtxt("07.1/gas/epot.dat", skiprows=1)
p_gas = np.loadtxt("07.1/gas/pres.dat", skiprows=1)


#Solid Phase
Ac_e_solid = np.zeros(times.size)
Ac_p_solid = np.zeros(times.size)

#print("Compute Autocorrelation Functions for the Solid phase")
#print("................")

for t in range(0, times.size):
    Ac_e_solid[t] = (1/(M-t))*(Cov_t(e_solid,M,t,1) - (1/(M-t))*mean_t(e_solid,M,t,0)*mean_t(e_solid,M,t,2))/((1/M)*Cov_t(e_solid,M,t,0) - ((1/M)*mean_t(e_solid,M,t,1))**2)
    Ac_p_solid[t] = (1/(M-t))*(Cov_t(p_solid,M,t,1) - (1/(M-t))*mean_t(p_solid,M,t,0)*mean_t(p_solid,M,t,2))/((1/M)*Cov_t(p_solid,M,t,0) - ((1/M)*mean_t(p_solid,M,t,1))**2)

#print("Done\n")
    

#Liquid Phase
Ac_e_liquid = np.zeros(times.size)
Ac_p_liquid = np.zeros(times.size)

#print("Compute Autocorrelation Functions for the Liquid phase")
#print("................")

for t in range(0, times.size):
    Ac_e_liquid[t] = (1/(M-t))*(Cov_t(e_liquid,M,t,1) - (1/(M-t))*mean_t(e_liquid,M,t,0)*mean_t(e_liquid,M,t,2))/((1/M)*Cov_t(e_liquid,M,t,0) - ((1/M)*mean_t(e_liquid,M,t,1))**2)
    Ac_p_liquid[t] = (1/(M-t))*(Cov_t(p_liquid,M,t,1) - (1/(M-t))*mean_t(p_liquid,M,t,0)*mean_t(p_liquid,M,t,2))/((1/M)*Cov_t(p_liquid,M,t,0) - ((1/M)*mean_t(p_liquid,M,t,1))**2)

#print("Done\n")
    

#Gas Phase
Ac_e_gas = np.zeros(times.size)
Ac_p_gas = np.zeros(times.size)

#print("Compute Autocorrelation Functions for the Gas phase")
#print("................")

for t in range(0, times.size):
    Ac_e_gas[t] = (1/(M-t))*(Cov_t(e_gas,M,t,1) - (1/(M-t))*mean_t(e_gas,M,t,0)*mean_t(e_gas,M,t,2))/((1/M)*Cov_t(e_gas,M,t,0) - ((1/M)*mean_t(e_gas,M,t,1))**2)
    Ac_p_gas[t] = (1/(M-t))*(Cov_t(p_gas,M,t,1) - (1/(M-t))*mean_t(p_gas,M,t,0)*mean_t(p_gas,M,t,2))/((1/M)*Cov_t(p_gas,M,t,0) - ((1/M)*mean_t(p_gas,M,t,1))**2)

#print("Done")

In [ ]:
#Call the fitting functions
#Solid phase
fit_e_solid = curve_fit(exp_fit, times, Ac_e_solid)
fit_e_solid_eq = fit_e_solid[0][0]*expit(-fit_e_solid[0][1]*times) + fit_e_solid[0][2]
fit_p_solid = curve_fit(exp_fit, times, Ac_p_solid)
fit_p_solid_eq = fit_p_solid[0][0]*expit(-fit_p_solid[0][1]*times) + fit_p_solid[0][2]

#Liquid phase
fit_e_liquid = curve_fit(exp_fit, times, Ac_e_liquid)
fit_e_liquid_eq = fit_e_liquid[0][0]*expit(-fit_e_liquid[0][1]*times) + fit_e_liquid[0][2]
fit_p_liquid = curve_fit(exp_fit, times, Ac_p_liquid)
fit_p_liquid_eq = fit_p_liquid[0][0]*expit(-fit_p_liquid[0][1]*times) + fit_p_liquid[0][2]

#Gas phase
fit_e_gas = curve_fit(exp_fit, times, Ac_e_gas)
fit_e_gas_eq = fit_e_gas[0][0]*expit(-fit_e_gas[0][1]*times) + fit_e_gas[0][2]
fit_p_gas = curve_fit(exp_fit, times, Ac_p_gas)
fit_p_gas_eq = fit_p_gas[0][0]*expit(-fit_p_gas[0][1]*times) + fit_p_gas[0][2]

In [ ]:
#plots
#Solid Phase
plt.figure(figsize=[70, 25])  #deafult (6.4,4.8)
plt.suptitle('Solid Phase', y=1.05, fontname = 'Bradley Hand', fontsize=110)

plt.subplot(1, 2, 1)
plt.title('Potential Energy autocorrelation', fontname = 'Bradley Hand', fontsize=90)
plt.xlabel('$t$', fontsize=80, labelpad=50)
plt.ylabel('$A_{C_{[\\dfrac{U}{N}]}}(t)$', fontsize=80, labelpad=50)
plt.xticks(fontsize=60)
plt.yticks(fontsize=60)
plt.xlim(0, 150)
plt.plot(times, Ac_e_solid, linewidth=5.5, linestyle='-', label='Monte Carlo')
plt.plot(times, fit_e_solid_eq, linewidth=5.5, linestyle='dashdot', color='olive',
         label='Exponential Fit $\\approx a e^{-b t} + c$')
plt.axvline(x=(1/fit_e_solid[0][1]), color='r', linewidth=3.5, linestyle='--', dashes=(10, 5),
            label='$ ( \\tau_c )_{fit} \sim  $' + str((int)(1/fit_e_solid[0][1])))
plt.axvline(x=2*(1/fit_e_solid[0][1]), color='orange', linewidth=3.5, linestyle='--', dashes=(10, 5),
            label='$ ( 2\\tau_c )_{fit} \sim  $' + str((int)(2*1/fit_e_solid[0][1])))
plt.legend(fontsize=60, loc='best')

plt.subplot(1, 2, 2)
plt.title('Pressure autocorrelation', fontname = 'Bradley Hand', fontsize=90)
plt.xlabel('$t$', fontsize=80, labelpad=50)
plt.ylabel('$A_{C_{[P]}}(t)$', fontsize=80, labelpad=50)
plt.xticks(fontsize=60)
plt.yticks(fontsize=60)
plt.xlim(0, 150)
plt.plot(times, Ac_p_solid, linewidth=5.5, linestyle='-', label='Monte Carlo')
plt.plot(times, fit_p_solid_eq, linewidth=5.5, linestyle='dashdot', color='olive',
         label='Exponential Fit $\\approx a e^{-b t} + c$')
plt.axvline(x=(1/fit_p_solid[0][1]), color='r', linewidth=3.5, linestyle='--', dashes=(10, 5),
            label='$ ( \\tau_c )_{fit} \sim  $' + str((int)(1/fit_p_solid[0][1])))
plt.axvline(x=2*(1/fit_p_solid[0][1]), color='orange', linewidth=3.5, linestyle='--', dashes=(10, 5),
            label='$ ( 2\\tau_c )_{fit} \sim  $' + str((int)(2*1/fit_p_solid[0][1])))
plt.legend(fontsize=60, loc='best')

plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.3, hspace=0.3)
plt.show()


#Liquid Phase
print("\n\n")
plt.figure(figsize=[70, 25])  #deafult (6.4,4.8)
plt.suptitle('Liquid Phase', y=1.05, fontname = 'Bradley Hand', fontsize=110)

plt.subplot(1, 2, 1)
plt.title('Potential Energy autocorrelation', fontname = 'Bradley Hand', fontsize=90)
plt.xlabel('$t$', fontsize=80, labelpad=50)
plt.ylabel('$A_{C_{[\\dfrac{U}{N}]}}(t)$', fontsize=80, labelpad=50)
plt.xticks(fontsize=60)
plt.yticks(fontsize=60)
plt.xlim(0, 300)
plt.plot(times, Ac_e_liquid, linewidth=5.5, linestyle='-', label='Monte Carlo')
plt.plot(times, fit_e_liquid_eq, linewidth=5.5, linestyle='dashdot', color='olive',
         label='Exponential Fit $\\approx a e^{-b t} + c$')
plt.axvline(x=(1/fit_e_liquid[0][1]), color='r', linewidth=3.5, linestyle='--', dashes=(10, 5),
            label='$ ( \\tau_c )_{fit} \sim  $' + str((int)(1/fit_e_liquid[0][1])))
plt.axvline(x=2*(1/fit_e_liquid[0][1]), color='orange', linewidth=3.5, linestyle='--', dashes=(10, 5),
            label='$ ( 2\\tau_c )_{fit} \sim  $' + str((int)(2*1/fit_e_liquid[0][1])))
plt.legend(fontsize=60, loc='best')

plt.subplot(1, 2, 2)
plt.title('Pressure autocorrelation', fontname = 'Bradley Hand', fontsize=90)
plt.xlabel('$t$', fontsize=80, labelpad=50)
plt.ylabel('$A_{C_{[P]}}(t)$', fontsize=80, labelpad=50)
plt.xticks(fontsize=60)
plt.yticks(fontsize=60)
plt.xlim(0, 300)
plt.plot(times, Ac_p_liquid, linewidth=5.5, linestyle='-', label='Monte Carlo')
plt.plot(times, fit_p_liquid_eq, linewidth=5.5, linestyle='dashdot', color='olive',
         label='Exponential Fit $\\approx a e^{-b t} + c$')
plt.axvline(x=(1/fit_p_liquid[0][1]), color='r', linewidth=3.5, linestyle='--', dashes=(10, 5),
            label='$ ( \\tau_c )_{fit} \sim  $' + str((int)(1/fit_p_liquid[0][1])))
plt.axvline(x=2*(1/fit_p_liquid[0][1]), color='orange', linewidth=3.5, linestyle='--', dashes=(10, 5),
            label='$ ( 2\\tau_c )_{fit} \sim  $' + str((int)(2*1/fit_p_liquid[0][1])))
plt.legend(fontsize=60, loc='best')

plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.3, hspace=0.3)
plt.show()


#Gas Phase
print("\n\n")
plt.figure(figsize=[70, 25])  #deafult (6.4,4.8)
plt.suptitle('Gas Phase', y=1.05, fontname = 'Bradley Hand', fontsize=110)

plt.subplot(1, 2, 1)
plt.title('Potential Energy autocorrelation', fontname = 'Bradley Hand', fontsize=90)
plt.xlabel('$t$', fontsize=80, labelpad=50)
plt.ylabel('$A_{C_{[\\dfrac{U}{N}]}}(t)$', fontsize=80, labelpad=50)
plt.xticks(fontsize=60)
plt.yticks(fontsize=60)
plt.xlim(0, 40)
plt.plot(times, Ac_e_gas, linewidth=5.5, linestyle='-', label='Monte Carlo')
plt.plot(times, fit_e_gas_eq, linewidth=5.5, linestyle='dashdot', color='olive',
         label='Exponential Fit $\\approx a e^{-b t} + c$')
plt.axvline(x=(1/fit_e_gas[0][1]), color='r', linewidth=3.5, linestyle='--', dashes=(10, 5),
            label='$ ( \\tau_c )_{fit} \sim  $' + str((int)(1/fit_e_gas[0][1])))
plt.axvline(x=2*(1/fit_e_gas[0][1]), color='orange', linewidth=3.5, linestyle='--',  dashes=(10, 5),
            label='$ ( 2\\tau_c )_{fit} \sim  $' + str((int)(2*1/fit_e_gas[0][1])))
plt.legend(fontsize=60, loc='best')

plt.subplot(1, 2, 2)
plt.title('Pressure autocorrelation', fontname = 'Bradley Hand', fontsize=90)
plt.xlabel('$t$', fontsize=80, labelpad=50)
plt.ylabel('$A_{C_{[P]}}(t)$', fontsize=80, labelpad=50)
plt.xticks(fontsize=60)
plt.yticks(fontsize=60)
plt.xlim(0, 40)
plt.plot(times, Ac_p_gas, linewidth=5.5, linestyle='-', label='Monte Carlo')
plt.plot(times, fit_p_gas_eq, linewidth=5.5, linestyle='dashdot', color='olive',
         label='Exponential Fit $\\approx a e^{-b t} + c$')
plt.axvline(x=(1/fit_p_gas[0][1]), color='r', linewidth=3.5, linestyle='--', dashes=(10, 5),
            label='$ ( \\tau_c )_{fit} \sim  $' + str((int)(1/fit_p_gas[0][1])))
plt.axvline(x=2*(1/fit_p_gas[0][1]), color='orange', linewidth=3.5, linestyle='--', dashes=(10, 5),
            label='$ ( 2\\tau_c )_{fit} \sim  $' + str((int)(2*1/fit_p_gas[0][1])))
plt.legend(fontsize=60, loc='best')

plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.3, hspace=0.3)
plt.show()

The data produced by the simulations clearly show a rapid decay of the autocorrelation functions, both for the potential energy and for the pressure, in all the three phases; in fact I only need to show a few hundred times in the plots to see the correlations drop to zero, in particular in the case of the gas phase. <br>
So, consequently, I am certain to deal with configurations uncorrelated to those that generated them during the stochastic chain by choosing blocks much longer than the correlation length that I have just estimated, i.e. a few hundred MC-steps, allowing the proper use of the blocking technique and *CLT*. <br>
To further investigate the choice regarding the length of the blocks I use the same data to study the estimation of the statistical uncertainties of the expectation value of $ \dfrac{U}{N} $ and $ P $ for different size of the blocks in the data blocking method, from $ L = \dfrac{M}{N_{blk}} = 10 $ to $ L = 5 \times 10^4 $, being $ N_{blk} $ the number of blocks. <br>
I carry out this analysis for all the phases of the Argon system also in this case and plot the values of the estimated uncertainties for the last average value of each blocking technique as a function of $ L $.

In [ ]:
#############################
#Blocking Method
#different number of blocks
#############################
#different choices for 
#the number of blocks to use
N_blk = np.array([5*10**4, 10**4, 5000, 1000, 500, 100, 50, 10])
L = np.floor_divide(M, N_blk)  #length of each block
                               #from L=M/N=10 to L=M/N=5*10^4
x = np.zeros(N_blk.size, dtype=object)


#Blocking variables
e_ave_solid = np.zeros(N_blk.size, dtype=object)   #average value of energy and pressure
p_ave_solid = np.zeros(N_blk.size, dtype=object)   #with a particular number of
e_ave_liquid = np.zeros(N_blk.size, dtype=object)  #blocks used to divide M Monte Carlo steps
p_ave_liquid = np.zeros(N_blk.size, dtype=object)  
e_ave_gas = np.zeros(N_blk.size, dtype=object)
p_ave_gas = np.zeros(N_blk.size, dtype=object)

e_ave2_solid = np.zeros(N_blk.size, dtype=object)  #Estimation of the error
p_ave2_solid = np.zeros(N_blk.size, dtype=object)
err_e_solid = np.zeros(N_blk.size, dtype=object)
err_p_solid = np.zeros(N_blk.size, dtype=object)
e_ave2_liquid = np.zeros(N_blk.size, dtype=object)
p_ave2_liquid = np.zeros(N_blk.size, dtype=object)
err_e_liquid = np.zeros(N_blk.size, dtype=object)
err_p_liquid = np.zeros(N_blk.size, dtype=object)
e_ave2_gas = np.zeros(N_blk.size, dtype=object)
p_ave2_gas = np.zeros(N_blk.size, dtype=object)
err_e_gas = np.zeros(N_blk.size, dtype=object)
err_p_gas = np.zeros(N_blk.size, dtype=object)
for j in range(N_blk.size):
    x[j] = np.arange(N_blk[N_blk.size-(j+1)])
    

#Solid Phase
#Blocking Method
#print("Compute Statistical Uncertainties for the Solid phase")
#print("................")

for N in range(N_blk.size):
    #print("Start Blocking Method ", N)
    #print("Using ", N_blk[N], "blocks")
    
    e_ave_solid[N] = np.zeros(N_blk[N])
    e_ave2_solid[N] = np.zeros(N_blk[N])
    err_e_solid[N] = np.zeros(N_blk[N])
    e_blk = np.zeros(N_blk[N])
    p_ave_solid[N] = np.zeros(N_blk[N])
    p_ave2_solid[N] = np.zeros(N_blk[N])
    err_p_solid[N] = np.zeros(N_blk[N])
    p_blk = np.zeros(N_blk[N])
    for j in range(N_blk[N]):  #loop over the number of
                               #blocks in the particular 
                               #Blocking Method selected
        for l in range(L[N]):  #loop over the elements
                               #of each single block
            e_blk[j] += e_solid[j*L[N]+l]
            p_blk[j] += p_solid[j*L[N]+l]
        e_blk[j] /= L[N]
        p_blk[j] /= L[N]
            
    for k in range(N_blk[N]):
        for m in range(k+1):
            #accumulate block averages
            e_ave_solid[N][k] += e_blk[m]
            p_ave_solid[N][k] += p_blk[m]
            #accumulate block averages^2
            e_ave2_solid[N][k] += e_blk[m]**2
            p_ave2_solid[N][k] += p_blk[m]**2
        #calculate the averages 
        #and the corresponding errors
        e_ave_solid[N][k] /= (k+1) 
        e_ave2_solid[N][k] /= (k+1)      
        err_e_solid[N][k] = error(e_ave_solid[N], e_ave2_solid[N], k)
        p_ave_solid[N][k] /= (k+1) 
        p_ave2_solid[N][k] /= (k+1)      
        err_p_solid[N][k] = error(p_ave_solid[N], p_ave2_solid[N], k)
        
    #print("End Blocking Method ", N, "\n")
    
#print("Done\n")


#Liquid Phase
#Blocking Method
#print("Compute Statistical Uncertainties for the Liquid phase")
#print("................")

for N in range(N_blk.size):
    #print("Start Blocking Method ", N)
    #print("Using ", N_blk[N], "blocks")
    
    e_ave_liquid[N] = np.zeros(N_blk[N])
    e_ave2_liquid[N] = np.zeros(N_blk[N])
    err_e_liquid[N] = np.zeros(N_blk[N])
    e_blk = np.zeros(N_blk[N])
    p_ave_liquid[N] = np.zeros(N_blk[N])
    p_ave2_liquid[N] = np.zeros(N_blk[N])
    err_p_liquid[N] = np.zeros(N_blk[N])
    p_blk = np.zeros(N_blk[N])
    for j in range(N_blk[N]):  #loop over the number of
                               #blocks in the particular 
                               #Blocking Method selected
        for l in range(L[N]):  #loop over the elements
                               #of each single block
            e_blk[j] += e_liquid[j*L[N]+l]
            p_blk[j] += p_liquid[j*L[N]+l]
        e_blk[j] /= L[N]
        p_blk[j] /= L[N]
            
    for k in range(N_blk[N]):
        for m in range(k+1):
            #accumulate block averages
            e_ave_liquid[N][k] += e_blk[m]
            p_ave_liquid[N][k] += p_blk[m]
            #accumulate block averages^2
            e_ave2_liquid[N][k] += e_blk[m]**2
            p_ave2_liquid[N][k] += p_blk[m]**2
        #calculate the averages 
        #and the corresponding errors
        e_ave_liquid[N][k] /= (k+1) 
        e_ave2_liquid[N][k] /= (k+1)      
        err_e_liquid[N][k] = error(e_ave_liquid[N], e_ave2_liquid[N], k)
        p_ave_liquid[N][k] /= (k+1) 
        p_ave2_liquid[N][k] /= (k+1)      
        err_p_liquid[N][k] = error(p_ave_liquid[N], p_ave2_liquid[N], k)
        
    #print("End Blocking Method ", N, "\n")
    
#print("Done\n")


#Gas Phase
#Blocking Method
#print("Compute Statistical Uncertainties for the Gas phase")
#print("................")

for N in range(N_blk.size):
    #print("Start Blocking Method ", N)
    #print("Using ", N_blk[N], "blocks")
    
    e_ave_gas[N] = np.zeros(N_blk[N])
    e_ave2_gas[N] = np.zeros(N_blk[N])
    err_e_gas[N] = np.zeros(N_blk[N])
    e_blk = np.zeros(N_blk[N])
    p_ave_gas[N] = np.zeros(N_blk[N])
    p_ave2_gas[N] = np.zeros(N_blk[N])
    err_p_gas[N] = np.zeros(N_blk[N])
    p_blk = np.zeros(N_blk[N])
    for j in range(N_blk[N]):  #loop over the number of
                               #blocks in the particular 
                               #Blocking Method selected
        for l in range(L[N]):  #loop over the elements
                               #of each single block
            e_blk[j] += e_gas[j*L[N]+l]
            p_blk[j] += p_gas[j*L[N]+l]
        e_blk[j] /= L[N]
        p_blk[j] /= L[N]
            
    for k in range(N_blk[N]):
        for m in range(k+1):
            #accumulate block averages
            e_ave_gas[N][k] += e_blk[m]
            p_ave_gas[N][k] += p_blk[m]
            #accumulate block averages^2
            e_ave2_gas[N][k] += e_blk[m]**2
            p_ave2_gas[N][k] += p_blk[m]**2
        #calculate the averages 
        #and the corresponding errors
        e_ave_gas[N][k] /= (k+1) 
        e_ave2_gas[N][k] /= (k+1)      
        err_e_gas[N][k] = error(e_ave_gas[N], e_ave2_gas[N], k)
        p_ave_gas[N][k] /= (k+1) 
        p_ave2_gas[N][k] /= (k+1)      
        err_p_gas[N][k] = error(p_ave_gas[N], p_ave2_gas[N], k)
        
    #print("End Blocking Method ", N, "\n")
    
#print("Done\n")

In [ ]:
#Solid Phase
E_solid = np.zeros(L.size)
E_err_solid = np.zeros(L.size)
P_solid = np.zeros(L.size)
P_err_solid = np.zeros(L.size)
for i in range(L.size):
    E_solid[i] = e_ave_solid[i][len(e_ave_solid[i])-1]
    E_err_solid[i] = err_e_solid[i][len(err_e_solid[i])-1]
    P_solid[i] = p_ave_solid[i][len(p_ave_solid[i])-1]
    P_err_solid[i] = err_p_solid[i][len(err_p_solid[i])-1]


#Liquid Phase
E_liquid = np.zeros(L.size)
E_err_liquid = np.zeros(L.size)
P_liquid = np.zeros(L.size)
P_err_liquid = np.zeros(L.size)
for i in range(L.size):
    E_liquid[i] = e_ave_liquid[i][len(e_ave_liquid[i])-1]
    E_err_liquid[i] = err_e_liquid[i][len(err_e_liquid[i])-1]
    P_liquid[i] = p_ave_liquid[i][len(p_ave_liquid[i])-1]
    P_err_liquid[i] = err_p_liquid[i][len(err_p_liquid[i])-1]
    
    
#Gas Phase
E_gas = np.zeros(L.size)
E_err_gas = np.zeros(L.size)
P_gas = np.zeros(L.size)
P_err_gas = np.zeros(L.size)
for i in range(L.size):
    E_gas[i] = e_ave_gas[i][len(e_ave_gas[i])-1]
    E_err_gas[i] = err_e_gas[i][len(err_e_gas[i])-1]
    P_gas[i] = p_ave_gas[i][len(p_ave_gas[i])-1]
    P_err_gas[i] = err_p_gas[i][len(err_p_gas[i])-1]

In [ ]:
plt.figure(figsize=[70, 25])  #deafult (6.4,4.8)

plt.subplot(1, 2, 1)
plt.title('Potential Energy Statistical Uncertainty', y=1.05, fontname = 'Bradley Hand', fontsize=90)
plt.xlabel('$L$', fontsize=80, labelpad=50)
plt.ylabel('$\sigma_{U/N}$', fontsize=80, labelpad=50)
plt.xticks(fontsize=60)
plt.yticks(fontsize=60)
plt.xscale('log')
plt.scatter(L, E_err_solid, color='olive', s=2.5*10**3, marker='^', label='Solid')
plt.scatter(L, E_err_liquid, color='blue', s=2.5*10**3, marker='H', label='liquid')
plt.scatter(L, E_err_gas, color='orange', s=2*10**3, marker='D', label='Gas')
plt.legend(fontsize=60, loc='best')

plt.subplot(1, 2, 2)
plt.title('Pressure Statistical Uncertainty', y=1.05, fontname = 'Bradley Hand', fontsize=90)
plt.xlabel('$L$', fontsize=80, labelpad=50)
plt.ylabel('$\sigma_{P}$', fontsize=80, labelpad=50)
plt.xticks(fontsize=60)
plt.yticks(fontsize=60)
plt.xscale('log')
plt.scatter(L, P_err_solid, color='olive', s=2.5*10**3, marker='^', label='Solid')
plt.scatter(L, P_err_liquid, color='blue', s=2.5*10**3, marker='H', label='liquid')
plt.scatter(L, P_err_gas, color='orange', s=2*10**3, marker='D', label='Gas')
plt.legend(fontsize=60, loc='best')

plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.3, hspace=0.3)
plt.show()

Note that the statistical uncertainty estimated through the block average is small by choosing small blocks, and gradually becomes larger by increasing the length of the block, converging to a plateau (which will be the correct one) in the range $ L \in [10^2, 10^4] $. <br>
To sum up: in each of the following Monte Carlo simulations I equilibrate (without measuring anything, but only generating new configurations) the system for $ N_{eq} = 200 $ MC-steps, and I estimate the average values of the various required quantities by dividing the total MC-time, which from now on is $ M = 10^5 $, in $ N_{blk} = 100 $ blocks of length $ L = 10^3 $.<br>
Now I'm ready to start!

## <font color='blue'>Exercise 07.2</font>
I include in the provided code the calculation of the *pair (or radial) distribution function* $ g(r) $ with $ r \in \big[0, \frac{l}{2} \big] $ being the distance between a pair of particles and $ l $ the edge of the simulation box, which gives the probability of finding a pair of atoms a distance $ r $ apart, relative to the probability expected for a completely random distribution (*ideal gas*) at the same density. <br>
This is an easy task by using the following algorithmic formula:

$$ 
  g(r) = \dfrac{1}{\rho N_{part} \Delta V(r)} \ \bigg \langle 
  \sum_{i=1}^{N_{part}} \sum_{j \neq i = 1}^{N_{part}} \delta \Big( 
  \vert \vec{r} \vert - \vert \vec{r}_i - \vec{r}_j \vert
  \Big) 
  \bigg \rangle
$$

with $ \Delta V(r) = \dfrac{4 \pi}{3} \ \Big[ (r+dr)^3 - r^3 \Big]$. In fact, I divide $ \big[0, \frac{l}{2} \big] $ into $ 100 $ subintervals (*bins*) $ \Big\{ \big[r_i, r_i + dr\big] \Big\}_{i=1}^{100} $ of length $ dr = \dfrac{l}{200} $ and I count how many particles are at a distance between $ r_i $ and $ r_i+dr $ for each of the configurations generated during the simulation: every time I find a pair of particles in a certain interval $ \big[ r_i, r_i+dr \big] $  I increase the value of the corresponding histogram bin by $ 2 $. I estimate $ \bigg \langle \sum_{i=1}^{N_{part}} \sum_{j \neq i = 1}^{N_{part}} \delta \Big( \vert \vec{r} \vert - \vert \vec{r}_i - \vec{r}_j \vert \Big) \bigg \rangle $ making the block average on the configurations generated and, reminding the normalization, $ g(r) $ with its uncertainty as a result. <br>

In [ ]:
###############################
#Calculation of the
#Radial Distribution Function
#g(r) with r ∈ [0, L/2]
###############################
#load the data
nbins = 100


#Solid Phase
r_solid = np.zeros(nbins)
bins_solid, gofr_solid, err_gofr_solid = np.loadtxt("07.2/solid/output.gave.0", usecols=(0, 1, 2), unpack=True)

#taking r as the mid point of each bin
for j in range(nbins):
    if j == 0:
        r_solid[j] = 0
    else:
        r_solid[j] = (bins_solid[j] + bins_solid[j-1])/2


#Liquid Phase
r_liquid = np.zeros(nbins)
bins_liquid, gofr_liquid, err_gofr_liquid = np.loadtxt("07.2/liquid/output.gave.0", usecols=(0, 1, 2), unpack=True)

#taking r as the mid point of each bin
for j in range(nbins):
    if j == 0:
        r_liquid[j] = 0
    else:
        r_liquid[j] = (bins_liquid[j] + bins_liquid[j-1])/2


#Gas Phase
r_gas = np.zeros(nbins)
bins_gas, gofr_gas, err_gofr_gas = np.loadtxt("07.2/gas/output.gave.0", usecols=(0, 1, 2), unpack=True)

#taking r as the mid point of each bin
for j in range(nbins):
    if j == 0:
        r_gas[j] = 0
    else:
        r_gas[j] = (bins_gas[j] + bins_gas[j-1])/2

In [ ]:
#plots
#Solid Phase
plt.figure(figsize=[70, 25])  #deafult (6.4,4.8)
plt.suptitle('Solid Phase', y=1.05, fontname = 'Bradley Hand', fontsize=110)

plt.subplot(1, 2, 1)
plt.title('Normal y scale', y=1.04, fontname = 'Bradley Hand', fontsize=90)
plt.xlabel('$r^{\star}$', fontsize=90, labelpad=50)
plt.ylabel('$g(r^{\star})$', fontsize=90, labelpad=50)
plt.xticks(fontsize=55)
plt.yticks(fontsize=55)
plt.xlim(0, 2.5)
plt.errorbar(r_solid, gofr_solid, yerr=err_gofr_solid, fmt='o', color='olive',
             elinewidth=10, markersize=20, capsize=22)
plt.axhline(y=1.0, color='r', linewidth=4.5, linestyle='--', dashes=(10, 5), label='Ideal Gas')
plt.legend(fontsize=65, loc='best')
plt.grid(True)

plt.subplot(1, 2, 2)
plt.title('Logarithmic y scale', y=1.04, fontname = 'Bradley Hand', fontsize=90)
plt.xlabel('$r^{\star}$', fontsize=90, labelpad=50)
plt.ylabel('$g(r^{\star})$', fontsize=90, labelpad=50)
plt.xticks(fontsize=55)
plt.yticks(fontsize=55)
plt.xlim(0, 2.5)
plt.yscale('log')
plt.errorbar(r_solid, gofr_solid, yerr=err_gofr_solid, fmt='o', color='olive',
             elinewidth=10, markersize=20, capsize=22)
plt.axhline(y=1.0, color='r', linewidth=4.5, linestyle='--', dashes=(10, 5), label='Ideal Gas')
plt.legend(fontsize=65, loc='lower right')
plt.grid(True)

plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.3, hspace=0.3)
plt.show()
print('\n\n')

In [ ]:
#Liquid Phase
plt.figure(figsize=[70, 25])  #deafult (6.4,4.8)
plt.suptitle('Liquid Phase', y=1.05, fontname = 'Bradley Hand', fontsize=110)

plt.subplot(1, 2, 1)
plt.title('Normal y scale', y=1.04, fontname = 'Bradley Hand', fontsize=90)
plt.xlabel('$r^{\star}$', fontsize=90, labelpad=50)
plt.ylabel('$g(r^{\star})$', fontsize=90, labelpad=50)
plt.xticks(fontsize=55)
plt.yticks(fontsize=55)
plt.xlim(0, 3)
plt.errorbar(r_liquid, gofr_liquid, yerr=err_gofr_liquid, fmt='o', color='blue',
             elinewidth=10, markersize=20, capsize=22)
plt.axhline(y=1.0, color='r', linewidth=4.5, linestyle='--', dashes=(10, 5), label='Ideal Gas')
plt.legend(fontsize=65, loc='best')
plt.grid(True)

plt.subplot(1, 2, 2)
plt.title('Logarithmic y scale', y=1.04, fontname = 'Bradley Hand', fontsize=90)
plt.xlabel('$r^{\star}$', fontsize=90, labelpad=50)
plt.ylabel('$g(r^{\star})$', fontsize=90, labelpad=50)
plt.xticks(fontsize=55)
plt.yticks(fontsize=55)
plt.xlim(0, 3)
plt.yscale('log')
plt.errorbar(r_liquid, gofr_liquid, yerr=err_gofr_liquid, fmt='o', color='blue',
             elinewidth=10, markersize=20, capsize=22)
plt.axhline(y=1.0, color='r', linewidth=4.5, linestyle='--', dashes=(10, 5), label='Ideal Gas')
plt.legend(fontsize=65, loc='lower right')
plt.grid(True)

plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.3, hspace=0.3)
plt.show()
print('\n\n')

In [ ]:
#Gas Phase
plt.figure(figsize=[70, 25])  #deafult (6.4,4.8)
plt.suptitle('Gas Phase', y=1.05, fontname = 'Bradley Hand', fontsize=110)

plt.subplot(1, 2, 1)
plt.title('Normal y scale', y=1.04, fontname = 'Bradley Hand', fontsize=90)
plt.xlabel('$r^{\star}$', fontsize=90, labelpad=50)
plt.ylabel('$g(r^{\star})$', fontsize=90, labelpad=50)
plt.xticks(fontsize=55)
plt.yticks(fontsize=55)
plt.xlim(0, 6.5)
plt.errorbar(r_gas, gofr_gas, yerr=err_gofr_gas, fmt='o', color='orange',
             elinewidth=10, markersize=20, capsize=22)
plt.axhline(y=1.0, color='r', linewidth=4.5, linestyle='--', dashes=(10, 5), label='Ideal Gas')
plt.legend(fontsize=65, loc='best')
plt.grid(True)

plt.subplot(1, 2, 2)
plt.title('Logarithmic y scale', y=1.04, fontname = 'Bradley Hand', fontsize=90)
plt.xlabel('$r^{\star}$', fontsize=90, labelpad=50)
plt.ylabel('$g(r^{\star})$', fontsize=90, labelpad=50)
plt.xticks(fontsize=55)
plt.yticks(fontsize=55)
plt.xlim(0, 6.5)
plt.yscale('log')
plt.errorbar(r_gas, gofr_gas, yerr=err_gofr_gas, fmt='o', color='orange',
             elinewidth=10, markersize=20, capsize=22)
plt.axhline(y=1.0, color='r', linewidth=4.5, linestyle='--', dashes=(10, 5), label='Ideal Gas')
plt.legend(fontsize=65, loc='lower right')
plt.grid(True)

plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.3, hspace=0.3)
plt.show()

The Metropolis algorithm never samples particles below a distance of approximately one $ \sigma $ ($ r^{\star} = 1.0 $) in each of the three phases, and this is due to the core part of the Lennard-Jones potential to which the particles are subjected; above this distance instead the radial distribution function reveals the different configurational structures of the three phases: in the solid phase I have regular, periodic structures, with particles fluctuating near their lattice positions and in fact discrete peaks at values of around $ \sigma $, $ \sqrt{2} \sigma, \sqrt{3} \sigma $, etc, can be seen. Each of this peak has a broadened shape which is caused by particles vibrating around their lattice sites. There is a near-zero probability of finding a particle in regions between these peaks as all atoms are packed regularly to fill the space most efficiently.<br>
The liquid phase shows concentric shells instead, with a first sharp peak at about one $ \sigma $ and subsequent peaks which occur roughly in intervals of $ \sigma $ but be much smaller than the first; due to their ability to move dynamically liquids do not maintain a constant structure and lose all of their long-range structure: in fact it can be easily seen from the plots above that at great distances the peaks oscillations dampen to $ 1 $, i.e. the liquid at long distances is indistinguishable from a disordered ideal gas phase (<font color='red'>red dotted line</font>). <br>
Finally the radial distribution function of the gas phase has only a single peak which rapidly decay to the ideal gas density $ g(r^{\star}) = 1 $ (<font color='red'>red dotted line</font>): in this case the Argon particles do not have a regular structure and this heavily influences the $ g(r) $. 

## <font color='blue'>Exercise 07.3</font>
Now I simply add the same algorithm for the calculation of $ g(r) $ to the **Molecular Dynamics (MD)** code (<font color='orange'>Numerical Exercise 4</font>); I remember that in this case the configurations of the system are obtained by solving the *Newton Equations* of motion.  <br>
I compare the MD results with those obtained in the previous exercise (<font color='magenta'>dashdot magenta line</font>).<br>
I must point out that the results of the simulations requested in this exercise but related to the MD code are contained in files saved in the folder <font color='olive'>Optional Exercise</font> of the <font color='orange'>Numerical Exercise 4</font>; moreover these results have been obtained choosing $ N_{blk} = 10^2 $ blocks so that a proper comparison can be made. In this context, it is also crucial to equilibrate the MD system at the right average temperature specified in the parameters of the three phases above.

In [ ]:
###############################
#Calculation of the
#Radial Distribution Function
#g(r) with r ∈ [0, L/2]
#Molecular Dynamics
###############################
#load the data
#Solid Phase
r_solid_MD = np.zeros(nbins)
bins_solid_MD, gofr_solid_MD, err_gofr_solid_MD = np.loadtxt("../Exercise 04/Optional Exercise/solid/ave_gofr.dat", usecols=(0, 1, 2), unpack=True)

#taking r as the mid point of each bin
for j in range(nbins):
    if j == 0:
        r_solid_MD[j] = 0
    else:
        r_solid_MD[j] = (bins_solid_MD[j] + bins_solid_MD[j-1])/2


#Liquid Phase
r_liquid_MD = np.zeros(nbins)
bins_liquid_MD, gofr_liquid_MD, err_gofr_liquid_MD = np.loadtxt("../Exercise 04/Optional Exercise/liquid/ave_gofr.dat", usecols=(0, 1, 2), unpack=True)

#taking r as the mid point of each bin
for j in range(nbins):
    if j == 0:
        r_liquid_MD[j] = 0
    else:
        r_liquid_MD[j] = (bins_liquid_MD[j] + bins_liquid_MD[j-1])/2


#Gas Phase
r_gas_MD = np.zeros(nbins)
bins_gas_MD, gofr_gas_MD, err_gofr_gas_MD = np.loadtxt("../Exercise 04/Optional Exercise/gas/ave_gofr.dat", usecols=(0, 1, 2), unpack=True)

#taking r as the mid point of each bin
for j in range(nbins):
    if j == 0:
        r_gas_MD[j] = 0
    else:
        r_gas_MD[j] = (bins_gas_MD[j] + bins_gas_MD[j-1])/2

In [ ]:
#plots
#Solid Phase
plt.figure(figsize=[70, 25])  #deafult (6.4,4.8)
plt.suptitle('Solid Phase', y=1.05, fontname = 'Bradley Hand', fontsize=110)

plt.subplot(1, 2, 1)
plt.title('Normal y scale', y=1.04, fontname = 'Bradley Hand', fontsize=90)
plt.xlabel('$r^{\star}$', fontsize=90, labelpad=50)
plt.ylabel('$g(r^{\star})$', fontsize=90, labelpad=50)
plt.xticks(fontsize=55)
plt.yticks(fontsize=55)
plt.xlim(0, 2.5)
plt.errorbar(r_solid_MD, gofr_solid_MD, yerr=err_gofr_solid_MD, fmt='o', color='olive',
             elinewidth=10, markersize=20, capsize=22, label='MD NVE')
plt.plot(r_solid, gofr_solid, color='magenta', linewidth=3.5, linestyle='dashdot', label='MC NVT')
plt.legend(fontsize=65, loc='best')
plt.grid(True)

plt.subplot(1, 2, 2)
plt.title('Logarithmic y scale', y=1.04, fontname = 'Bradley Hand', fontsize=90)
plt.xlabel('$r^{\star}$', fontsize=90, labelpad=50)
plt.ylabel('$g(r^{\star})$', fontsize=90, labelpad=50)
plt.xticks(fontsize=55)
plt.yticks(fontsize=55)
plt.xlim(0, 2.5)
plt.yscale('log')
plt.errorbar(r_solid_MD, gofr_solid_MD, yerr=err_gofr_solid_MD, fmt='o', color='olive',
             elinewidth=10, markersize=20, capsize=22, label='MD NVE')
plt.plot(r_solid, gofr_solid, color='magenta', linewidth=3.5, linestyle='dashdot', label='MC NVT')
plt.grid(True)
plt.legend(fontsize=65, loc='lower right')

plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.3, hspace=0.3)
plt.show()
print('\n\n')

In [ ]:
#Liquid Phase
plt.figure(figsize=[70, 25])  #deafult (6.4,4.8)
plt.suptitle('Liquid Phase', y=1.05, fontname = 'Bradley Hand', fontsize=110)

plt.subplot(1, 2, 1)
plt.title('Normal y scale', y=1.04, fontname = 'Bradley Hand', fontsize=90)
plt.xlabel('$r^{\star}$', fontsize=90, labelpad=50)
plt.ylabel('$g(r^{\star})$', fontsize=90, labelpad=50)
plt.xticks(fontsize=55)
plt.yticks(fontsize=55)
plt.xlim(0, 3)
plt.errorbar(r_liquid_MD, gofr_liquid_MD, yerr=err_gofr_liquid_MD, fmt='o', color='blue',
             elinewidth=10, markersize=20, capsize=22, label='MD NVE')
plt.plot(r_liquid, gofr_liquid, color='magenta', linewidth=3.5, linestyle='dashdot', label='MC NVT')
plt.legend(fontsize=65, loc='best')
plt.grid(True)

plt.subplot(1, 2, 2)
plt.title('Logarithmic y scale', y=1.04, fontname = 'Bradley Hand', fontsize=90)
plt.xlabel('$r^{\star}$', fontsize=90, labelpad=50)
plt.ylabel('$g(r^{\star})$', fontsize=90, labelpad=50)
plt.xticks(fontsize=55)
plt.yticks(fontsize=55)
plt.xlim(0, 3)
plt.yscale('log')
plt.errorbar(r_liquid_MD, gofr_liquid_MD, yerr=err_gofr_liquid_MD, fmt='o', color='blue',
             elinewidth=10, markersize=20, capsize=22, label='MD NVE')
plt.plot(r_liquid, gofr_liquid, color='magenta', linewidth=3.5, linestyle='dashdot', label='MC NVT')
plt.legend(fontsize=65, loc='lower right')
plt.grid(True)

plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.3, hspace=0.3)
plt.show()
print('\n\n')

In [ ]:
#Gas Phase
plt.figure(figsize=[70, 25])  #deafult (6.4,4.8)
plt.suptitle('Gas Phase', y=1.05, fontname = 'Bradley Hand', fontsize=110)

plt.subplot(1, 2, 1)
plt.title('Normal y scale', y=1.04, fontname = 'Bradley Hand', fontsize=90)
plt.xlabel('$r^{\star}$', fontsize=90, labelpad=50)
plt.ylabel('$g(r^{\star})$', fontsize=90, labelpad=50)
plt.xticks(fontsize=55)
plt.yticks(fontsize=55)
plt.xlim(0, 6.5)
plt.errorbar(r_gas_MD, gofr_gas_MD, yerr=err_gofr_gas_MD, fmt='o', color='orange',
             elinewidth=10, markersize=20, capsize=22, label='MD NVE')
plt.plot(r_gas, gofr_gas, color='magenta', linewidth=3.5, linestyle='dashdot', label='MC NVT')
plt.legend(fontsize=65, loc='best')
plt.grid(True)

plt.subplot(1, 2, 2)
plt.title('Logarithmic y scale', y=1.04, fontname = 'Bradley Hand', fontsize=90)
plt.xlabel('$r^{\star}$', fontsize=90, labelpad=50)
plt.ylabel('$g(r^{\star})$', fontsize=90, labelpad=50)
plt.xticks(fontsize=55)
plt.yticks(fontsize=55)
plt.xlim(0, 6.5)
plt.yscale('log')
plt.errorbar(r_gas_MD, gofr_gas_MD, yerr=err_gofr_gas_MD, fmt='o', color='orange',
             elinewidth=10, markersize=20, capsize=22, label='MD NVE')
plt.plot(r_gas, gofr_gas, color='magenta', linewidth=3.5, linestyle='dashdot', label='MC NVT')
plt.legend(fontsize=65, loc='best')
plt.grid(True)

plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.3, hspace=0.3)
plt.show()

The plots just shown highlight an excellent agreement between the results obtained through integration of the equations of motion (MD) and those relating to the use of *ensamble theory* in *Statistical Mechanics* (MC).<br>
Already from here it is possible to guess the power of stochastic sampling, being the Metropolis algorithm able to reconstruct in a consistent way the structure of the three phases of a classical many-particles system through the sampling of the Boltzmann weight.

## <font color='blue'>Exercise 07.4</font>

To conclude the simulation of this statistical system I compute average values and uncertainties for the potential energy per particle, the pressure and the radial distribution function in **SI units** and compare this *Monte Carlo NVT* results with those obtained with *Molecular Dynamics NVE* simulations (<font color='orange'>Numerical Exercise 4</font> -> <font color='olive'>Optional Exercise </font>) in similar thermodynamic conditions.

<div class="alert alert-block alert-info">
 
  <b>Note:</b> <br>
  Also in the Monte Carlo simulations I choose to use as initial configuration 
  (<font face='Courier'>config.0</font>) for the solid phase that in which the atoms are organized according to 
  an *fcc lattice*, while for both the liquid and the gaseous phase I use the last configuration generated in 
  the simulation of the solid: it will be the task of the equilibration routine to melt the system at the 
  correct target temperature.
    
</div>

In [ ]:
#Units of measure
#LJ --> SI
SIGMA=0.34*10**(-9)  #[𝜎] = m unit of length
EPS=120  #[𝜖/𝑘b] = K unit of temperature
Kb=1.38064852*10**(-23)  #[Kb] = m^2 kg s^(-2) K^(-1)
M=6.6335209*10**(-26)  #[m] = kg --> m=39.948 amu
unit_of_energy=EPS*Kb;  #Joule [J]
unit_of_time=SIGMA*sqrt(M/unit_of_energy);  #seconds [s]
unit_of_pressure=unit_of_energy/(SIGMA**3);  #Pascal [Pa]

N_blk=np.loadtxt("07.4/solid/output.epot.0", usecols=0)

In [ ]:
###################
#Monte Carlo NVT
###################
#load the data
epot_solid_MC, err_epot_solid_MC = np.loadtxt("07.4/solid/output.epot.0", usecols=(2, 3), unpack=True)
pres_solid_MC, err_pres_solid_MC = np.loadtxt("07.4/solid/output.pres.0", usecols=(2, 3), unpack=True)
epot_liquid_MC, err_epot_liquid_MC = np.loadtxt("07.4/liquid/output.epot.0", usecols=(2, 3), unpack=True)
pres_liquid_MC, err_pres_liquid_MC = np.loadtxt("07.4/liquid/output.pres.0", usecols=(2, 3), unpack=True)
epot_gas_MC, err_epot_gas_MC = np.loadtxt("07.4/gas/output.epot.0", usecols=(2, 3), unpack=True)
pres_gas_MC, err_pres_gas_MC = np.loadtxt("07.4/gas/output.pres.0", usecols=(2, 3), unpack=True)


########################
#Molecular Dynamics NVE
########################
#load the data
epot_solid_MD, err_epot_solid_MD = np.loadtxt("../Exercise 04/Optional Exercise/solid/ave_epot.dat",
                                              usecols=(0, 1), unpack=True)
pres_solid_MD, err_pres_solid_MD = np.loadtxt("../Exercise 04/Optional Exercise/solid/ave_pres.dat",
                                              usecols=(0, 1), unpack=True)
epot_liquid_MD, err_epot_liquid_MD = np.loadtxt("../Exercise 04/Optional Exercise/liquid/ave_epot.dat",
                                                usecols=(0, 1), unpack=True)
pres_liquid_MD, err_pres_liquid_MD = np.loadtxt("../Exercise 04/Optional Exercise/liquid/ave_pres.dat",
                                                usecols=(0, 1), unpack=True)
epot_gas_MD, err_epot_gas_MD = np.loadtxt("../Exercise 04/Optional Exercise/gas/ave_epot.dat",
                                          usecols=(0, 1), unpack=True)
pres_gas_MD, err_pres_gas_MD = np.loadtxt("../Exercise 04/Optional Exercise/gas/ave_pres.dat",
                                          usecols=(0, 1), unpack=True)

In [ ]:
#####################
#plots
#####################
#Solid Phase
plt.figure(figsize=[70, 55]) #deafult (6.4,4.8)
plt.suptitle('Solid Phase', fontname = 'Bradley Hand', y=1.03, fontsize=120)

###
#Potential Energy
###
plt1=plt.subplot(2, 2, 1)
plt.title('Potential Energy', fontname = 'Bradley Hand', y=1.04, fontsize=100)
plt.xlabel('$N_{blk}$', fontsize=90, labelpad=50)
plt.ylabel('$\dfrac{U}{N} \ [J]$', fontsize=90, labelpad=50)
plt.xticks(fontsize=55)
plt.yticks(fontsize=55)
plt1.ticklabel_format(style='sci', useMathText=True)
plt1.yaxis.get_offset_text().set_fontsize(55)
plt.errorbar(N_blk, unit_of_energy*epot_solid_MC, yerr=unit_of_energy*err_epot_solid_MC, fmt='o',
             elinewidth=3.0, color='orange', markersize=12, capsize=16, label='MC NVT')
plt.errorbar(N_blk, unit_of_energy*epot_solid_MD, yerr=unit_of_energy*err_epot_solid_MD, fmt='o',
             elinewidth=3.0, color='blue', markersize=12, capsize=16, label='MD NVE')
plt.grid(True)
plt.legend(fontsize=65, loc='best')

###
#Pressure
###
plt2=plt.subplot(2, 2, 2)
plt.title('Pressure', fontname = 'Bradley Hand', y=1.04, fontsize=100)
plt.xlabel('$N_{blk}$', fontsize=90, labelpad=50)
plt.ylabel('$P \ [Pa]$', fontsize=90, labelpad=50)
plt.xticks(fontsize=55)
plt.yticks(fontsize=55)
plt2.ticklabel_format(style='sci', useMathText=True)
plt2.yaxis.get_offset_text().set_fontsize(55)
plt.errorbar(N_blk, unit_of_pressure*pres_solid_MC, yerr=err_pres_solid_MC*unit_of_pressure, fmt='o',
             elinewidth=3.0, color='orange', markersize=12, capsize=16, label='MC NVT')
plt.errorbar(N_blk, unit_of_pressure*pres_solid_MD, yerr=err_pres_solid_MD*unit_of_pressure, fmt='o',
             elinewidth=3.0, color='blue', markersize=12, capsize=16, label='MD NVE')
plt.grid(True)
plt.legend(fontsize=65, loc='best')

###
#Radial Distribution Function
###
plt3=plt.subplot(2, 2, 3)
plt.title('Radial Distribution Function (Normal y scale)', y=1.04, fontname = 'Bradley Hand', fontsize=100)
plt.xlabel('$r \ [m]$', fontsize=90, labelpad=50)
plt.ylabel('$g(r)$', fontsize=90, labelpad=50)
plt.xticks(fontsize=55)
plt.yticks(fontsize=55)
plt3.ticklabel_format(style='sci', useMathText=True)
plt3.xaxis.get_offset_text().set_fontsize(55)
plt.errorbar(r_solid*SIGMA, gofr_solid, yerr=err_gofr_solid, fmt='o', color='orange',
             elinewidth=10, markersize=20, capsize=22, label='MC NVT')
plt.errorbar(r_solid_MD*SIGMA, gofr_solid_MD, yerr=err_gofr_solid_MD, fmt='o', color='blue',
             alpha=0.4, elinewidth=10, markersize=20, capsize=22, label='MD NVE')
plt.grid(True)
plt.legend(fontsize=65, loc='best')

###
#Radial Distribution Function
###
plt4=plt.subplot(2, 2, 4)
plt.title('Radial Distribution Function (Logarithmic y scale)', y=1.04, fontname = 'Bradley Hand', fontsize=100)
plt.xlabel('$r \ [m]$', fontsize=90, labelpad=50)
plt.ylabel('$g(r)$', fontsize=90, labelpad=50)
plt.xticks(fontsize=55)
plt.yticks(fontsize=55)
plt4.ticklabel_format(style='sci', useMathText=True)
plt4.xaxis.get_offset_text().set_fontsize(55)
plt.yscale('log')
plt.errorbar(r_solid*SIGMA, gofr_solid, yerr=err_gofr_solid, fmt='o', color='orange',
             elinewidth=10, markersize=20, capsize=22, label='MC NVT')
plt.errorbar(r_solid_MD*SIGMA, gofr_solid_MD, yerr=err_gofr_solid_MD, fmt='o', color='blue',
             alpha=0.4, elinewidth=10, markersize=20, capsize=22, label='MD NVE')
plt.grid(True)
plt.legend(fontsize=65, loc='best')

plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.4, hspace=0.4)
plt.show()
print('\n\n')

In [ ]:
#Liquid Phase
plt.figure(figsize=[70, 55]) #deafult (6.4,4.8)
plt.suptitle('Liquid Phase', fontname = 'Bradley Hand', y=1.03, fontsize=120)

###
#Potential Energy
###
plt1=plt.subplot(2, 2, 1)
plt.title('Potential Energy', fontname = 'Bradley Hand', y=1.04, fontsize=100)
plt.xlabel('$N_{blk}$', fontsize=90, labelpad=50)
plt.ylabel('$\dfrac{U}{N} \ [J]$', fontsize=90, labelpad=50)
plt.xticks(fontsize=55)
plt.yticks(fontsize=55)
plt1.ticklabel_format(style='sci', useMathText=True)
plt1.yaxis.get_offset_text().set_fontsize(55)
plt.errorbar(N_blk, unit_of_energy*epot_liquid_MC, yerr=err_epot_liquid_MC*unit_of_energy, fmt='o', elinewidth=3.0,
             color='orange', markersize=12, capsize=16, label='MC NVT')
plt.errorbar(N_blk, epot_liquid_MD*unit_of_energy, yerr=err_epot_liquid_MD*unit_of_energy, fmt='o', elinewidth=3.0,
             color='blue', markersize=12, capsize=16, label='MD NVE')
plt.grid(True)
plt.legend(fontsize=65, loc='best')

###
#Pressure
###
plt2=plt.subplot(2, 2, 2)
plt.title('Pressure', fontname = 'Bradley Hand', y=1.04, fontsize=100)
plt.xlabel('$N_{blk}$', fontsize=90, labelpad=50)
plt.ylabel('$P \ [Pa]$', fontsize=90, labelpad=50)
plt.xticks(fontsize=55)
plt.yticks(fontsize=55)
plt2.ticklabel_format(style='sci', useMathText=True)
plt2.yaxis.get_offset_text().set_fontsize(55)
plt.errorbar(N_blk, unit_of_pressure*pres_liquid_MC, yerr=err_pres_liquid_MC*unit_of_pressure, fmt='o',
             elinewidth=3.0, color='orange', markersize=12, capsize=16, label='MC NVT')
plt.errorbar(N_blk, pres_liquid_MD*unit_of_pressure, yerr=err_pres_liquid_MD*unit_of_pressure, fmt='o',
             elinewidth=3.0, color='blue', markersize=12, capsize=16, label='MD NVE')
plt.grid(True)
plt.legend(fontsize=65, loc='best')

###
#Radial Distribution Function
###
plt3=plt.subplot(2, 2, 3)
plt.title('Radial Distribution Function (Normal y scale)', y=1.04, fontname = 'Bradley Hand', fontsize=100)
plt.xlabel('$r \ [m]$', fontsize=90, labelpad=50)
plt.ylabel('$g(r)$', fontsize=90, labelpad=50)
plt.xticks(fontsize=55)
plt.yticks(fontsize=55)
plt3.ticklabel_format(style='sci', useMathText=True)
plt3.xaxis.get_offset_text().set_fontsize(55)
plt.errorbar(r_liquid*SIGMA, gofr_liquid, yerr=err_gofr_liquid, fmt='o', color='orange',
             elinewidth=10, markersize=20, capsize=22, label='MC NVT')
plt.errorbar(r_liquid_MD*SIGMA, gofr_liquid_MD, yerr=err_gofr_liquid_MD, fmt='o', color='blue',
             alpha=0.4, elinewidth=10, markersize=20, capsize=22, label='MD NVE')
plt.grid(True)
plt.legend(fontsize=65, loc='best')

###
#Radial Distribution Function
###
plt4=plt.subplot(2, 2, 4)
plt.title('Radial Distribution Function (Logarithmic y scale)', y=1.04, fontname = 'Bradley Hand', fontsize=100)
plt.xlabel('$r \ [m]$', fontsize=90, labelpad=50)
plt.ylabel('$g(r)$', fontsize=90, labelpad=50)
plt.xticks(fontsize=55)
plt.yticks(fontsize=55)
plt4.ticklabel_format(style='sci', useMathText=True)
plt4.xaxis.get_offset_text().set_fontsize(55)
plt.yscale('log')
plt.errorbar(r_liquid*SIGMA, gofr_liquid, yerr=err_gofr_liquid, fmt='o', color='orange',
             elinewidth=10, markersize=20, capsize=22, label='MC NVT')
plt.errorbar(r_liquid_MD*SIGMA, gofr_liquid_MD, yerr=err_gofr_liquid_MD, fmt='o', color='blue',
             alpha=0.4, elinewidth=10, markersize=20, capsize=22, label='MD NVE')
plt.grid(True)
plt.legend(fontsize=65, loc='best')

plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.4, hspace=0.4)
plt.show()
print('\n\n')

In [ ]:
#Gas Phase
plt.figure(figsize=[70, 55]) #deafult (6.4,4.8)
plt.suptitle('Gas Phase', fontname = 'Bradley Hand', y=1.03, fontsize=120)

###
#Potential Energy
###
plt1=plt.subplot(2, 2, 1)
plt.title('Potential Energy', fontname = 'Bradley Hand', y=1.04, fontsize=100)
plt.xlabel('$N_{blk}$', fontsize=90, labelpad=50)
plt.ylabel('$\dfrac{U}{N} \ [J]$', fontsize=90, labelpad=50)
plt.xticks(fontsize=55)
plt.yticks(fontsize=55)
plt1.ticklabel_format(style='sci', useMathText=True)
plt1.yaxis.get_offset_text().set_fontsize(55)
plt.errorbar(N_blk, epot_gas_MC*unit_of_energy, yerr=err_epot_gas_MC*unit_of_energy, fmt='o',
             elinewidth=3.0, color='orange', markersize=12, capsize=16, label='MC NVT')
plt.errorbar(N_blk, epot_gas_MD*unit_of_energy, yerr=err_epot_gas_MD*unit_of_energy, fmt='o',
             elinewidth=3.0, color='blue', markersize=12, capsize=16, label='MD NVE')
plt.grid(True)
plt.legend(fontsize=65, loc='best')

###
#Pressure
###
plt2=plt.subplot(2, 2, 2)
plt.title('Pressure', fontname = 'Bradley Hand', y=1.04, fontsize=100)
plt.xlabel('$N_{blk}$', fontsize=90, labelpad=50)
plt.ylabel('$P \ [Pa]$', fontsize=90, labelpad=50)
plt.xticks(fontsize=55)
plt.yticks(fontsize=55)
plt2.ticklabel_format(style='sci', useMathText=True)
plt2.yaxis.get_offset_text().set_fontsize(55)
plt.errorbar(N_blk, pres_gas_MC*unit_of_pressure, yerr=err_pres_gas_MC*unit_of_pressure, fmt='o',
             elinewidth=3.0, color='orange', markersize=12, capsize=16, label='MC NVT')
plt.errorbar(N_blk, pres_gas_MD*unit_of_pressure, yerr=err_pres_gas_MD*unit_of_pressure, fmt='o',
             elinewidth=3.0, color='blue', markersize=12, capsize=16, label='MD NVE')
plt.grid(True)
plt.legend(fontsize=65, loc='best')

###
#Radial Distribution Function
###
plt3=plt.subplot(2, 2, 3)
plt.title('Radial Distribution Function (Normal y scale)', y=1.04, fontname = 'Bradley Hand', fontsize=100)
plt.xlabel('$r \ [m]$', fontsize=90, labelpad=50)
plt.ylabel('$g(r)$', fontsize=90, labelpad=50)
plt.xticks(fontsize=55)
plt.yticks(fontsize=55)
plt3.ticklabel_format(style='sci', useMathText=True)
plt3.xaxis.get_offset_text().set_fontsize(55)
plt.errorbar(r_gas*SIGMA, gofr_gas, yerr=err_gofr_gas, fmt='o', color='orange',
             alpha=0.4, elinewidth=10, markersize=20, capsize=22, label='MC NVT')
plt.errorbar(r_gas_MD*SIGMA, gofr_gas_MD, yerr=err_gofr_gas_MD, fmt='o', color='blue',
             alpha=0.4, elinewidth=10, markersize=20, capsize=22, label='MD NVE')
plt.grid(True)
plt.legend(fontsize=65, loc='best')

###
#Radial Distribution Function
###
plt4=plt.subplot(2, 2, 4)
plt.title('Radial Distribution Function (Logarithmic y scale)', y=1.04, fontname = 'Bradley Hand', fontsize=100)
plt.xlabel('$r \ [m]$', fontsize=90, labelpad=50)
plt.ylabel('$g(r)$', fontsize=90, labelpad=50)
plt.xticks(fontsize=55)
plt.yticks(fontsize=55)
plt4.ticklabel_format(style='sci', useMathText=True)
plt4.xaxis.get_offset_text().set_fontsize(55)
plt.yscale('log')
plt.errorbar(r_gas*SIGMA, gofr_gas, yerr=err_gofr_gas, fmt='o', color='orange',
             alpha=0.4, elinewidth=10, markersize=20, capsize=22, label='MC NVT')
plt.errorbar(r_gas_MD*SIGMA, gofr_gas_MD, yerr=err_gofr_gas_MD, fmt='o', color='blue',
             alpha=0.4, elinewidth=10, markersize=20, capsize=22, label='MD NVE')
plt.grid(True)
plt.legend(fontsize=65, loc='best')

plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.4, hspace=0.4)
plt.show()

This comparison brings out a series of considerations; the first is that the systems I have simulated clearly show not perfectly equal thermodynamic conditions (relative to the same phase), even if similar anyway. This aspect is influenced by the equilibration phase of the algorithm, which in my opinion is much more difficult in the case of Molecular Dynamics than in the case of Monte Carlo: in the first technique the target temperature never comes into play explicitly as a fixed parameter, except in the calculation of the velocities scale factor, and indeed one of the aims of the code is precisely to determine the temperature as a function of the number of blocks (see <font color='orange'>Numerical Exercise 4</font>); in the case of the Metropolis algorithm temperature plays a fundamental role instead, appearing in the exponent of the Boltzmann weight which driving the Markov chain of the system configurations. This different importance of the temperature inside the two algorithms can generate greater difficulty in equilibrating the systems in the same thermodynamic condition, in particular in the solid phase, and this involves the differences in the estimates of the thermodynamic quantities shown above. <br>

The values I obtain for the pressure of the three phases are very different from each other, and this can be traced precisely to the explicit dependence of the temperature in the equation to calculate it

$$ 
   P = \rho k_B T + \frac{1}{3V} \left\langle \sum_{i=1}^{N-1} \sum_{j\,(>i)}^N 48\epsilon 
   \left[ \left(\frac{\sigma}{|\vec{r}_i -\vec{r}_j|}\right)^{12} - 
   \frac{1}{2} \left(\frac{\sigma}{|\vec{r}_i -\vec{r}_j|}\right)^6 \right] \right\rangle
$$

amplified by the use of the SI units of measure. Listed these aspects related to temperature, the results obtained with the two algorithms are of the same order of magnitude and very close with regard to potential energy. The difference between the width of the error bars in the two cases is due to an extremely more precise analysis of the correlations made earlier than in the case of Molecular Dynamics, where the values used for $ M $ and $ N_{blk} $ have been taken with reasonableness but not precision. <br>
As mentioned above, however, the (stastistic) equality between the two radial distribution functions in all three phases is very significant. <br>
In conclusion, it is undoubtedly possible to work on the equilibration phase in the MD algorithm to improve the discrepancy between the two thermodynamic conditions in the two different codes, but what emerges from this comparison is that Statistical Mechanics works!